# Свойства многофазного потока по uniflocvba и uniflocpy

In [ ]:
import sys
sys.path.append('../')

import uniflocpy.uTools.data_workflow as data_workflow
import pandas as pd
from uniflocpy.uTools import plotly_workflow
import uniflocpy.uPVT.BlackOil_model as BlackOil_model
import uniflocpy.uValidation.python_api as python_api
import uniflocpy.uTools.uconst as uc
import uniflocpy.uPVT.PVT_fluids as PVT_fluids

In [ ]:
def create_result_df_from_vba_output(vba_result, p_bar, t_c):
    result_dict = {}
    for key, value in zip(vba_result[1],vba_result[0]):
        result_dict[key] = value
    result_dict['p_bar'] = p_bar
    result_dict['t_c'] = t_c
    this_df = pd.DataFrame(result_dict, index = [0])
    return this_df

Загрузка надстройки

In [ ]:
uniflocvba = python_api.API('E:\\Git\\unifloc_vba\\UniflocVBA_7.xlam')

Задание параметров

In [ ]:
gamma_oil = 0.87
gamma_water = 1
gamma_gas = 1.2
rsb_m3m3 = 120
t_res_c = 100
t_c = 50
p_bar = 100
q_liq_sm3day = 100
watercut_perc = 20


keywords_python = {"gamma_oil": gamma_oil, "gamma_gas": gamma_gas, "gamma_wat":gamma_water,
                                    "rsb_m3m3": rsb_m3m3, "t_res_c": t_res_c}

keywords_vba = {"qliq_sm3day":q_liq_sm3day, "fw_perc":watercut_perc,
                "t_C": t_c, "gamma_gas": gamma_gas,
                "gamma_oil": gamma_oil, "gamma_wat": gamma_water, "rsb_m3m3": rsb_m3m3, "tres_C": t_res_c} #проверено, улетают свойства газа (из-за z), st oil-gas, mu_wate



Настройка модели флюида

In [ ]:
blackoil_option = BlackOil_model.BlackOil_option()
blackoil_option.b_wat_cor_number = 1
blackoil_option.mu_wat_cor_number = 1
blackoil_option.rho_wat_cor_number = 1
blackoil_option.z_cor_number = 1
blackoil_option.pseudocritical_temperature_cor_number = 1
blackoil_option.pseudocritical_pressure_cor_number = 1
python_fluid = BlackOil_model.Fluid(**keywords_python, option = blackoil_option)

Инициализация многофазного потока

In [ ]:
multiphase_flow = PVT_fluids.FluidFlow(python_fluid)
multiphase_flow.qliq_on_surface_m3day = q_liq_sm3day
multiphase_flow.fw_on_surface_perc  =watercut_perc

Расчет

In [ ]:
python_flow_model_db = data_workflow.Data()
python_flow_model_db.clear_data()
vba_result_df = None
for p_bar in range(2, 400, 10):
    multiphase_flow.calc(p_bar, t_c)
    p_atm = uc.bar2atm(p_bar)
    python_flow_model_db.get_data(multiphase_flow, object_name='python_flow_model')
    vba_result = uniflocvba.MF_all_mf(p_atma = p_atm, **keywords_vba)
    this_vba_result_df = create_result_df_from_vba_output(vba_result, p_bar, t_c)
    try:
        vba_result_df = vba_result_df.append(this_vba_result_df)
    except:
        vba_result_df = this_vba_result_df.copy()

Объединение результатов

In [ ]:
result_unifloc_python = python_flow_model_db.get_data_as_df()
result_unifloc_python.index = result_unifloc_python['python_flow_model.p_bar']
result_unifloc_python.index.name = 'p_bar'

vba_result_df.index = result_unifloc_python.index
vba_result_df.index.name = 'p_bar'
vba_result_df['mn_kgsec'] = vba_result_df['mw_kgsec'] + vba_result_df['mg_kgsec'] + vba_result_df['mo_kgsec']
vba_result_df = vba_result_df.add_prefix('vba_flow_model.')
all_result = result_unifloc_python.join(vba_result_df)

Построение графиков

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['mu'], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result, ['rho'], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["heatcap"], ["cliq_JkgC"]], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["sigma"]], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["kgsec"]], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

# Суммарный массовый расход смеси должен быть постоянным

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["q", 'oil']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["q", 'wat']], ['fl.','cor_number', 'mass_fraction', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [["q", 'gas']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['liq', 'm3day']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['mix', 'm3day']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  [['gas_fraction']], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  ['fw'], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')

In [ ]:
plotly_workflow.plot_by_patterns(all_result,  ['fw'], ['fl.','cor_number', 'cal'], swap_xy=False, reversed_y=False,  plot_name='Распределение свойств ГЖС от давления')